In [1]:
%matplotlib inline

In [2]:
cd ..

C:\Projects\python\recommender


In [3]:
from functools import partial
from pathlib import Path
from datetime import datetime

import numpy as np
import pandas as pd
import torch as T
import torch.optim as optim

from utils import get_log_dir

In [4]:
from datasets import TorchStackOverflow
from models import FMLearner, TorchFM, TorchHrmFM, TorchPrmeFM, TorchTransFM
from models.fm_learner import simple_loss, trans_loss

In [5]:
DEVICE = T.cuda.current_device()
BATCH = 3000
SHUFFLE = True
WORKERS = 0
item_path = Path("./inputs/stackoverflow/item.csv")

In [6]:
db = TorchStackOverflow(data_path=item_path, min_user=4)
db

Original user size: 75572
Filtered user size: 14795
Original item dataframe shape: (299845, 8)
Filtered item dataframe shape: (217223, 8)


In [7]:
db.config_db(batch_size=BATCH,
             shuffle=SHUFFLE,
             num_workers=WORKERS,
             device=DEVICE)

## Create Criterion

In [8]:
# regst setting
LINEAR_REG = 1
EMB_REG = 1
TRANS_REG = 1

In [9]:
simple_loss_callback = partial(simple_loss, LINEAR_REG, EMB_REG)
simple_loss_callback

functools.partial(<function simple_loss at 0x000001B91F6AAA60>, 1, 1)

In [10]:
trans_loss_callback = partial(trans_loss, LINEAR_REG, EMB_REG, TRANS_REG)
trans_loss_callback

functools.partial(<function trans_loss at 0x000001B91F704B70>, 1, 1, 1)

## Train model

### Hyper-parameter

In [11]:
feat_dim = db.feat_dim
NUM_DIM = 124
INIT_MEAN = 0.1

## Train fm model

In [14]:
LEARNING_RATE = 0.001
DECAY_FREQ = 1000
DECAY_GAMMA = 1

In [13]:
fm_model = TorchFM(feature_dim=feat_dim, num_dim=NUM_DIM, init_mean=INIT_MEAN)
fm_model

TorchFM()

In [14]:
adam_opt = optim.Adam(fm_model.parameters(), lr=LEARNING_RATE)
schedular = optim.lr_scheduler.StepLR(adam_opt, step_size=DECAY_FREQ, gamma=DECAY_GAMMA)

In [15]:
fm_learner = FMLearner(fm_model, adam_opt, schedular, db)
fm_learner

In [16]:
fm_learner.fit(epoch=8, loss_callback=simple_loss_callback, log_dir=get_log_dir('fm'))

  0%|                                                                                                                                                                  | 0/8 [00:00<?, ?it/s]

Epoch: 0
Epoch 0 step 0: training loss: 157572.1662900254
Epoch 0 step 1: training accuarcy: 0.30966666666666665
Epoch 0 step 1: training loss: 152803.85492885328
Epoch 0 step 2: training accuarcy: 0.36633333333333334
Epoch 0 step 2: training loss: 148239.5385377525
Epoch 0 step 3: training accuarcy: 0.416
Epoch 0 step 3: training loss: 143949.19111709765
Epoch 0 step 4: training accuarcy: 0.43766666666666665
Epoch 0 step 4: training loss: 139690.8817278023
Epoch 0 step 5: training accuarcy: 0.48266666666666663
Epoch 0 step 5: training loss: 135535.64838294886
Epoch 0 step 6: training accuarcy: 0.5173333333333333
Epoch 0 step 6: training loss: 131506.05680230522
Epoch 0 step 7: training accuarcy: 0.5696666666666667
Epoch 0 step 7: training loss: 127644.8764366617
Epoch 0 step 8: training accuarcy: 0.5793333333333334
Epoch 0 step 8: training loss: 123925.68895784377
Epoch 0 step 9: training accuarcy: 0.6046666666666667
Epoch 0 step 9: training loss: 120230.83929059228
Epoch 0 step 10: t

 12%|███████████████████▎                                                                                                                                      | 1/8 [00:49<05:44, 49.19s/it]

Epoch: 1
Epoch 1 step 63: training loss: 20883.365795839152
Epoch 1 step 64: training accuarcy: 0.9
Epoch 1 step 64: training loss: 20124.462548850322
Epoch 1 step 65: training accuarcy: 0.9146666666666666
Epoch 1 step 65: training loss: 19443.443708299234
Epoch 1 step 66: training accuarcy: 0.9096666666666666
Epoch 1 step 66: training loss: 18777.48370399148
Epoch 1 step 67: training accuarcy: 0.9166666666666666
Epoch 1 step 67: training loss: 18151.625813766914
Epoch 1 step 68: training accuarcy: 0.9126666666666666
Epoch 1 step 68: training loss: 17540.09873886304
Epoch 1 step 69: training accuarcy: 0.9129999999999999
Epoch 1 step 69: training loss: 16938.938998389403
Epoch 1 step 70: training accuarcy: 0.9116666666666666
Epoch 1 step 70: training loss: 16354.709566784144
Epoch 1 step 71: training accuarcy: 0.9173333333333333
Epoch 1 step 71: training loss: 15805.788902313036
Epoch 1 step 72: training accuarcy: 0.9109999999999999
Epoch 1 step 72: training loss: 15276.489009289911
Epo

 25%|██████████████████████████████████████▌                                                                                                                   | 2/8 [01:38<04:55, 49.18s/it]

...

 75%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌                                      | 6/8 [04:53<01:37, 48.81s/it]

Epoch: 6
Epoch 6 step 378: training loss: 134.39942405316273
Epoch 6 step 379: training accuarcy: 0.9966666666666666
Epoch 6 step 379: training loss: 143.93534600999658
Epoch 6 step 380: training accuarcy: 0.994
Epoch 6 step 380: training loss: 142.43819564081656
Epoch 6 step 381: training accuarcy: 0.996
Epoch 6 step 381: training loss: 145.87698197705072
Epoch 6 step 382: training accuarcy: 0.993
Epoch 6 step 382: training loss: 139.60891512436964
Epoch 6 step 383: training accuarcy: 0.9963333333333333
Epoch 6 step 383: training loss: 149.25016637481065
Epoch 6 step 384: training accuarcy: 0.9926666666666666
Epoch 6 step 384: training loss: 142.18873267434103
Epoch 6 step 385: training accuarcy: 0.9946666666666666
Epoch 6 step 385: training loss: 141.72630611537724
Epoch 6 step 386: training accuarcy: 0.995
Epoch 6 step 386: training loss: 141.51432784574277
Epoch 6 step 387: training accuarcy: 0.9933333333333333
Epoch 6 step 387: training loss: 139.68929663926764
Epoch 6 step 388: t

 88%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊                   | 7/8 [05:42<00:48, 48.84s/it]

Epoch: 7
Epoch 7 step 441: training loss: 126.93586574471033
Epoch 7 step 442: training accuarcy: 0.9946666666666666
Epoch 7 step 442: training loss: 116.24161802498695
Epoch 7 step 443: training accuarcy: 0.9966666666666666
Epoch 7 step 443: training loss: 122.3233376558658
Epoch 7 step 444: training accuarcy: 0.9963333333333333
Epoch 7 step 444: training loss: 120.98224268883112
Epoch 7 step 445: training accuarcy: 0.9956666666666666
Epoch 7 step 445: training loss: 121.67969765727265
Epoch 7 step 446: training accuarcy: 0.997
Epoch 7 step 446: training loss: 128.5335223681605
Epoch 7 step 447: training accuarcy: 0.9936666666666666
Epoch 7 step 447: training loss: 114.32640492306439
Epoch 7 step 448: training accuarcy: 0.997
Epoch 7 step 448: training loss: 118.43309591184185
Epoch 7 step 449: training accuarcy: 0.9966666666666666
Epoch 7 step 449: training loss: 116.85175543654111
Epoch 7 step 450: training accuarcy: 0.9976666666666666
Epoch 7 step 450: training loss: 118.7723864524

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [06:32<00:00, 49.34s/it]


### Train HRM FM Model

In [23]:
hrm_model = TorchHrmFM(feature_dim=feat_dim, num_dim=NUM_DIM, init_mean=INIT_MEAN)
hrm_model

TorchHrmFM()

In [24]:
adam_opt = optim.Adam(hrm_model.parameters(), lr=LEARNING_RATE)
schedular = optim.lr_scheduler.StepLR(adam_opt,
                                      step_size=DECAY_FREQ,
                                      gamma=DECAY_GAMMA)

In [25]:
hrm_learner = FMLearner(hrm_model, adam_opt, schedular, db)
hrm_learner

In [26]:
hrm_learner.fit(epoch=8,
                loss_callback=simple_loss_callback,
                log_dir=get_log_dir('stackoverflow', 'hrm'))


  0%|                                                                                                                                                                  | 0/8 [00:00<?, ?it/s]

Epoch: 0
Epoch 0 step 0: training loss: 161424.97674656817
Epoch 0 step 1: training accuarcy: 0.5186666666666666
Epoch 0 step 1: training loss: 157219.70639475313
Epoch 0 step 2: training accuarcy: 0.4943333333333333
Epoch 0 step 2: training loss: 152539.92346734728
Epoch 0 step 3: training accuarcy: 0.5143333333333333
Epoch 0 step 3: training loss: 148512.27948702226
Epoch 0 step 4: training accuarcy: 0.495
Epoch 0 step 4: training loss: 143848.96650937246
Epoch 0 step 5: training accuarcy: 0.5053333333333333
Epoch 0 step 5: training loss: 139727.92820349155
Epoch 0 step 6: training accuarcy: 0.48566666666666664
Epoch 0 step 6: training loss: 135480.55833961524
Epoch 0 step 7: training accuarcy: 0.496
Epoch 0 step 7: training loss: 131344.59307532327
Epoch 0 step 8: training accuarcy: 0.49866666666666665
Epoch 0 step 8: training loss: 127493.79173152546
Epoch 0 step 9: training accuarcy: 0.49466666666666664
Epoch 0 step 9: training loss: 123907.64786004399
Epoch 0 step 10: training ac


 12%|███████████████████▎                                                                                                                                      | 1/8 [00:53<06:16, 53.75s/it]

Epoch: 1
Epoch 1 step 63: training loss: 20099.349826608475
Epoch 1 step 64: training accuarcy: 0.5393333333333333
Epoch 1 step 64: training loss: 19525.53089410358
Epoch 1 step 65: training accuarcy: 0.5236666666666666
Epoch 1 step 65: training loss: 18792.944452993535
Epoch 1 step 66: training accuarcy: 0.5313333333333333
Epoch 1 step 66: training loss: 18171.69781992276
Epoch 1 step 67: training accuarcy: 0.5356666666666666
Epoch 1 step 67: training loss: 17505.967809260328
Epoch 1 step 68: training accuarcy: 0.5376666666666666
Epoch 1 step 68: training loss: 16968.548029772355
Epoch 1 step 69: training accuarcy: 0.5266666666666666
Epoch 1 step 69: training loss: 16361.836175799031
Epoch 1 step 70: training accuarcy: 0.5509999999999999
Epoch 1 step 70: training loss: 15817.677259059481
Epoch 1 step 71: training accuarcy: 0.5406666666666666
Epoch 1 step 71: training loss: 15305.414113810642
Epoch 1 step 72: training accuarcy: 0.534
Epoch 1 step 72: training loss: 14814.313165580057
E


 25%|██████████████████████████████████████▌                                                                                                                   | 2/8 [01:46<05:21, 53.52s/it]

...


 75%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌                                      | 6/8 [05:20<01:47, 53.59s/it]

Epoch: 6
Epoch 6 step 378: training loss: 2011.4703182433225
Epoch 6 step 379: training accuarcy: 0.5976666666666667
Epoch 6 step 379: training loss: 1992.1870746287473
Epoch 6 step 380: training accuarcy: 0.606
Epoch 6 step 380: training loss: 1996.6260246726808
Epoch 6 step 381: training accuarcy: 0.6013333333333333
Epoch 6 step 381: training loss: 1997.9918530451678
Epoch 6 step 382: training accuarcy: 0.609
Epoch 6 step 382: training loss: 2007.524881895957
Epoch 6 step 383: training accuarcy: 0.5936666666666667
Epoch 6 step 383: training loss: 1992.2203747422484
Epoch 6 step 384: training accuarcy: 0.599
Epoch 6 step 384: training loss: 1989.8164323455285
Epoch 6 step 385: training accuarcy: 0.6016666666666667
Epoch 6 step 385: training loss: 2007.1022148668287
Epoch 6 step 386: training accuarcy: 0.5833333333333333
Epoch 6 step 386: training loss: 1983.1376794180271
Epoch 6 step 387: training accuarcy: 0.6053333333333333
Epoch 6 step 387: training loss: 2000.5422643004083
Epoch 6


 88%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊                   | 7/8 [06:13<00:53, 53.44s/it]

Epoch: 7
Epoch 7 step 441: training loss: 1999.778280009805
Epoch 7 step 442: training accuarcy: 0.5953333333333333
Epoch 7 step 442: training loss: 1994.0888515732663
Epoch 7 step 443: training accuarcy: 0.5926666666666667
Epoch 7 step 443: training loss: 2002.648652882715
Epoch 7 step 444: training accuarcy: 0.5906666666666667
Epoch 7 step 444: training loss: 1988.454796283186
Epoch 7 step 445: training accuarcy: 0.607
Epoch 7 step 445: training loss: 1993.1667836226102
Epoch 7 step 446: training accuarcy: 0.5913333333333333
Epoch 7 step 446: training loss: 1995.998275970917
Epoch 7 step 447: training accuarcy: 0.5953333333333333
Epoch 7 step 447: training loss: 1996.9595535129556
Epoch 7 step 448: training accuarcy: 0.6053333333333333
Epoch 7 step 448: training loss: 2012.9855808796176
Epoch 7 step 449: training accuarcy: 0.5893333333333333
Epoch 7 step 449: training loss: 2026.399698873912
Epoch 7 step 450: training accuarcy: 0.588
Epoch 7 step 450: training loss: 2002.681819573651


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [07:06<00:00, 53.09s/it]

### Train PRME FM Model

In [27]:
prme_model = TorchPrmeFM(feature_dim=feat_dim, num_dim=NUM_DIM, init_mean=INIT_MEAN)
prme_model

TorchPrmeFM()

In [28]:
adam_opt = optim.Adam(prme_model.parameters(), lr=LEARNING_RATE)
schedular = optim.lr_scheduler.StepLR(adam_opt,
                                      step_size=DECAY_FREQ,
                                      gamma=DECAY_GAMMA)

In [30]:
prme_learner = FMLearner(prme_model, adam_opt, schedular, db)
prme_learner

In [31]:
prme_learner.fit(epoch=8, loss_callback=simple_loss_callback, log_dir=get_log_dir('prme'))


  0%|                                                                                                                                                                  | 0/8 [00:00<?, ?it/s]

Epoch: 0
Epoch 0 step 0: training loss: 168569.2936286707
Epoch 0 step 1: training accuarcy: 0.4993333333333333
Epoch 0 step 1: training loss: 163485.7659552902
Epoch 0 step 2: training accuarcy: 0.4983333333333333
Epoch 0 step 2: training loss: 159378.10168334848
Epoch 0 step 3: training accuarcy: 0.5073333333333333
Epoch 0 step 3: training loss: 154501.4194061317
Epoch 0 step 4: training accuarcy: 0.5146666666666666
Epoch 0 step 4: training loss: 149650.16897265305
Epoch 0 step 5: training accuarcy: 0.5113333333333333
Epoch 0 step 5: training loss: 145888.4337116899
Epoch 0 step 6: training accuarcy: 0.5026666666666666
Epoch 0 step 6: training loss: 141368.1092794052
Epoch 0 step 7: training accuarcy: 0.5093333333333333
Epoch 0 step 7: training loss: 136958.64898813626
Epoch 0 step 8: training accuarcy: 0.5126666666666666
Epoch 0 step 8: training loss: 133736.63994669836
Epoch 0 step 9: training accuarcy: 0.5073333333333333
Epoch 0 step 9: training loss: 129658.23196107848
Epoch 0 st


 12%|███████████████████▎                                                                                                                                      | 1/8 [00:54<06:20, 54.32s/it]

Epoch: 1
Epoch 1 step 63: training loss: 25264.932058651277
Epoch 1 step 64: training accuarcy: 0.5313333333333333
Epoch 1 step 64: training loss: 24563.6291795637
Epoch 1 step 65: training accuarcy: 0.5233333333333333
Epoch 1 step 65: training loss: 23837.740697772657
Epoch 1 step 66: training accuarcy: 0.5263333333333333
Epoch 1 step 66: training loss: 23137.5456343006
Epoch 1 step 67: training accuarcy: 0.519
Epoch 1 step 67: training loss: 22305.188506839324
Epoch 1 step 68: training accuarcy: 0.5383333333333333
Epoch 1 step 68: training loss: 21865.87371189461
Epoch 1 step 69: training accuarcy: 0.5186666666666666
Epoch 1 step 69: training loss: 21126.914611093707
Epoch 1 step 70: training accuarcy: 0.5413333333333333
Epoch 1 step 70: training loss: 20507.082900245856
Epoch 1 step 71: training accuarcy: 0.5283333333333333
Epoch 1 step 71: training loss: 19900.68308730873
Epoch 1 step 72: training accuarcy: 0.5353333333333333
Epoch 1 step 72: training loss: 19344.148694673775
Epoch


 25%|██████████████████████████████████████▌                                                                                                                   | 2/8 [01:47<05:23, 53.96s/it]

...


 75%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌                                      | 6/8 [05:19<01:46, 53.35s/it]

Epoch: 6
Epoch 6 step 378: training loss: 2007.3039918269963
Epoch 6 step 379: training accuarcy: 0.6063333333333333
Epoch 6 step 379: training loss: 2014.3587262542972
Epoch 6 step 380: training accuarcy: 0.5936666666666667
Epoch 6 step 380: training loss: 2032.9607512739262
Epoch 6 step 381: training accuarcy: 0.5926666666666667
Epoch 6 step 381: training loss: 2023.6147372454552
Epoch 6 step 382: training accuarcy: 0.5916666666666667
Epoch 6 step 382: training loss: 2001.440187290323
Epoch 6 step 383: training accuarcy: 0.6113333333333333
Epoch 6 step 383: training loss: 2038.911427772406
Epoch 6 step 384: training accuarcy: 0.577
Epoch 6 step 384: training loss: 2042.6050643427182
Epoch 6 step 385: training accuarcy: 0.5856666666666667
Epoch 6 step 385: training loss: 2019.788289053153
Epoch 6 step 386: training accuarcy: 0.6026666666666667
Epoch 6 step 386: training loss: 2033.5953070702801
Epoch 6 step 387: training accuarcy: 0.588
Epoch 6 step 387: training loss: 2018.3627146785


 88%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊                   | 7/8 [06:12<00:53, 53.12s/it]

Epoch: 7
Epoch 7 step 441: training loss: 2020.356170581338
Epoch 7 step 442: training accuarcy: 0.5926666666666667
Epoch 7 step 442: training loss: 2017.4887357087612
Epoch 7 step 443: training accuarcy: 0.5933333333333333
Epoch 7 step 443: training loss: 1996.4614705658942
Epoch 7 step 444: training accuarcy: 0.6096666666666667
Epoch 7 step 444: training loss: 2010.033932801882
Epoch 7 step 445: training accuarcy: 0.599
Epoch 7 step 445: training loss: 2016.8442833552137
Epoch 7 step 446: training accuarcy: 0.5956666666666667
Epoch 7 step 446: training loss: 2015.6366536854407
Epoch 7 step 447: training accuarcy: 0.5863333333333333
Epoch 7 step 447: training loss: 2008.5217373952091
Epoch 7 step 448: training accuarcy: 0.5986666666666667
Epoch 7 step 448: training loss: 2006.8401407849979
Epoch 7 step 449: training accuarcy: 0.5993333333333333
Epoch 7 step 449: training loss: 2019.0411705627255
Epoch 7 step 450: training accuarcy: 0.5976666666666667
Epoch 7 step 450: training loss: 1


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [07:04<00:00, 52.95s/it]

### Train Trans FM Model

In [15]:
trans_model = TorchTransFM(feature_dim=feat_dim, num_dim=NUM_DIM, init_mean=INIT_MEAN)
trans_model

TorchTransFM()

In [16]:
adam_opt = optim.Adam(trans_model.parameters(), lr=LEARNING_RATE)
schedular = optim.lr_scheduler.StepLR(adam_opt,
                                      step_size=DECAY_FREQ,
                                      gamma=DECAY_GAMMA)

In [17]:
trans_learner = FMLearner(trans_model, adam_opt, schedular, db)
trans_learner

In [18]:
trans_learner.fit(epoch=8, loss_callback=trans_loss_callback, log_dir=get_log_dir('trans'))

  0%|                                                                                                                                                                  | 0/8 [00:00<?, ?it/s]

Epoch: 0
Epoch 0 step 0: training loss: 171935.10777800047
Epoch 0 step 1: training accuarcy: 0.4983333333333333
Epoch 0 step 1: training loss: 167236.04864530085
Epoch 0 step 2: training accuarcy: 0.5033333333333333
Epoch 0 step 2: training loss: 162522.13603857072
Epoch 0 step 3: training accuarcy: 0.506
Epoch 0 step 3: training loss: 158037.64368668137
Epoch 0 step 4: training accuarcy: 0.4943333333333333
Epoch 0 step 4: training loss: 153135.47970628904
Epoch 0 step 5: training accuarcy: 0.511
Epoch 0 step 5: training loss: 149167.705830932
Epoch 0 step 6: training accuarcy: 0.5096666666666666
Epoch 0 step 6: training loss: 144981.4878073405
Epoch 0 step 7: training accuarcy: 0.4993333333333333
Epoch 0 step 7: training loss: 140899.25312771543
Epoch 0 step 8: training accuarcy: 0.5026666666666666
Epoch 0 step 8: training loss: 136904.69567565294
Epoch 0 step 9: training accuarcy: 0.5086666666666666
Epoch 0 step 9: training loss: 132839.0323278288
Epoch 0 step 10: training accuarcy:

 12%|███████████████████▎                                                                                                                                      | 1/8 [00:57<06:39, 57.02s/it]

Epoch: 1
Epoch 1 step 63: training loss: 24765.28876517985
Epoch 1 step 64: training accuarcy: 0.5439999999999999
Epoch 1 step 64: training loss: 24343.362802849264
Epoch 1 step 65: training accuarcy: 0.533
Epoch 1 step 65: training loss: 23330.051831719284
Epoch 1 step 66: training accuarcy: 0.5363333333333333
Epoch 1 step 66: training loss: 22676.773948953753
Epoch 1 step 67: training accuarcy: 0.5393333333333333
Epoch 1 step 67: training loss: 21755.683702378883
Epoch 1 step 68: training accuarcy: 0.5483333333333333
Epoch 1 step 68: training loss: 21054.956681211257
Epoch 1 step 69: training accuarcy: 0.5553333333333333
Epoch 1 step 69: training loss: 20870.62700801407
Epoch 1 step 70: training accuarcy: 0.5376666666666666
Epoch 1 step 70: training loss: 19935.317096798874
Epoch 1 step 71: training accuarcy: 0.5493333333333333
Epoch 1 step 71: training loss: 19713.50639924015
Epoch 1 step 72: training accuarcy: 0.5473333333333333
Epoch 1 step 72: training loss: 18888.050174725067
Ep

 25%|██████████████████████████████████████▌                                                                                                                   | 2/8 [01:54<05:42, 57.03s/it]

...

 75%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌                                      | 6/8 [05:45<01:55, 57.58s/it]

Epoch: 6
Epoch 6 step 378: training loss: 2263.5935061250066
Epoch 6 step 379: training accuarcy: 0.6876666666666666
Epoch 6 step 379: training loss: 2287.9425524954586
Epoch 6 step 380: training accuarcy: 0.6923333333333334
Epoch 6 step 380: training loss: 2306.9840913737758
Epoch 6 step 381: training accuarcy: 0.695
Epoch 6 step 381: training loss: 2234.2039193461146
Epoch 6 step 382: training accuarcy: 0.7003333333333334
Epoch 6 step 382: training loss: 2269.0065718211335
Epoch 6 step 383: training accuarcy: 0.713
Epoch 6 step 383: training loss: 2270.095949674769
Epoch 6 step 384: training accuarcy: 0.6976666666666667
Epoch 6 step 384: training loss: 2308.040924046886
Epoch 6 step 385: training accuarcy: 0.6976666666666667
Epoch 6 step 385: training loss: 2332.956090578001
Epoch 6 step 386: training accuarcy: 0.6769999999999999
Epoch 6 step 386: training loss: 2311.3791655096375
Epoch 6 step 387: training accuarcy: 0.6913333333333334
Epoch 6 step 387: training loss: 2260.6101782034

 88%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊                   | 7/8 [06:43<00:57, 57.68s/it]

Epoch: 7
Epoch 7 step 441: training loss: 2165.3529943064427
Epoch 7 step 442: training accuarcy: 0.7133333333333333
Epoch 7 step 442: training loss: 2116.9963831491973
Epoch 7 step 443: training accuarcy: 0.719
Epoch 7 step 443: training loss: 2151.0668768045743
Epoch 7 step 444: training accuarcy: 0.7093333333333333
Epoch 7 step 444: training loss: 2132.4021095970916
Epoch 7 step 445: training accuarcy: 0.7253333333333333
Epoch 7 step 445: training loss: 2104.2357112608065
Epoch 7 step 446: training accuarcy: 0.7156666666666667
Epoch 7 step 446: training loss: 2126.812333236503
Epoch 7 step 447: training accuarcy: 0.721
Epoch 7 step 447: training loss: 2167.1015551817395
Epoch 7 step 448: training accuarcy: 0.7073333333333333
Epoch 7 step 448: training loss: 2180.3855493158503
Epoch 7 step 449: training accuarcy: 0.6996666666666667
Epoch 7 step 449: training loss: 2203.771525998578
Epoch 7 step 450: training accuarcy: 0.7096666666666667
Epoch 7 step 450: training loss: 2124.226691504

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [07:39<00:00, 57.32s/it]
